# Entrainer les agents addi

## Settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/permanent/RECHERCHE2/ml_flux_euler/

/content/drive/MyDrive/permanent/RECHERCHE2/ml_flux_euler


In [ ]:
#ou bien 
#!cp -r drive/MyDrive/permanent/RECHERCHE2/ml_flux_euler/* .

In [ ]:
import Euler.core_solver as core
import Euler.agent_addi as aga
import Euler.neural_networks as nn

from Euler.backend import K
from Euler.param import Param,Projecter
from Euler.initial_conditions import GenParam,FuncGenerator

#from Euler.backend import K
import matplotlib.pyplot as plt
import popup_lib.popup as pop
import numpy as np
import copy
import time

Popup loaded!


In [ ]:
order = 1
nx = 1200 // order
if order == 1:
  nx = 600
nx_ratio = nx // 100

param = Param(nx=nx,nx_ratio=nx_ratio,
              BC_solver=Param.BC_periodic,
              BC_model=Param.BC_periodic)

k_tf32=K("tf",32)

genParam=GenParam(param,kind=GenParam.kind_changing)
genParam.continuousPart_scale = 0.5
genParam.discontinuousPart_scale =0.5
genParam.sod_scale=0.5
genParam.same_discontinuity_locations=True
# genParam.initial_speed_factor= 1
genParam.initial_speed_factor= 0
genParam.changing_discountMinSpacing= 0.4
genParam.changing_discountMaxSpacing= 0.8
generator=FuncGenerator(genParam, param, k_tf32)

Param initialised with, nx=600, nx_coarse=99


In [ ]:
fam_size=6
period_duration="15 seconds" #avant accélération: 60 secondes
all_agents=[]
nb_t = 800

In [ ]:
## la première fois c'est plus long (traçage)
ti=time.time()
W_train=core.generate_fine_solutions(genParam,param, nb_t,k_tf32)
print("durée:",time.time()-ti)
W_train.shape

durée: 1.4795558452606201


TensorShape([800, 64, 600, 3])

## Définition des familles d'agent

In [ ]:
model_struct = (32,64,32)
agent= aga.Agent_addi(param, nn.Difference_model_tricky(param,5,model_struct=model_struct),
                            watch_duration=20,
                            lossCoef_stab=1., #10.
                            lossCoef_ridge=3e-3,
                            lossCoef_disHLL=1e-2, #1.
                            nb_optimization=15,
                            score_computed_last_time=True
                            )

In [ ]:
def family_full(window_size,color):
        name="full_"+str(window_size)
        agents=[]
        model_struct = (32,64,32)
        for _ in range(fam_size):
            agent= aga.Agent_addi(param, nn.Difference_model_tricky(param,window_size,model_struct=model_struct),
                            watch_duration=20,
                            lossCoef_stab=20., #10.
                            lossCoef_ridge=3e-3,
                            lossCoef_disHLL=1.,
                            nb_optimization=15,
                            score_computed_last_time=True
                            )
            agents.append(agent)
            all_agents.append(agent)
            
        return pop.Family_trainer(
            agents=agents,
            nb_bestweights_averaged=3,
            nb_strong=3,
            period_duration=period_duration,
            name=name,
            color=color)

family_trainers=[
                 family_full(5,"red"),
                #  family_full(3,"blue"),
                 ]

def load_data():
    nb_t=800
    ti=time.time()
    W_train = core.generate_fine_solutions(genParam,param, nb_t,k_tf32)
    W_val = core.generate_fine_solutions(genParam,param, nb_t,k_tf32)
    print(f"|génération des données, durée: {time.time()-ti:.2f} ",end="")
    
    ti=time.time()
    for agent in all_agents:
        agent.load_WY_train(W_train)
        agent.load_WY_valid(W_val)
    print(f"|load données,durée: {time.time()-ti:.2f}",end="")

In [ ]:
try:
    for i in range(50):
        load_data()
        for family_trainer in family_trainers:
            family_trainer.period()
except KeyboardInterrupt:
    for family_trainer in family_trainers:
        #pour pouvoir reprendre si on veut
        family_trainer.interupt_period()

|génération des données, durée: 0.91 traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
traçage de la méthode _load_WY_accelerated
|load données,durée: 9.27
full_5,échauffement Traçage de la méthode gradient_tape
Traçage de la méthode gradient_tape
traçage de la méthode _test_accelerated, pour l'estimation 'is_model?':

### Historique des `famparams`

In [ ]:
def plot_history_famparams():
    # keys=["lossCoef_stab","lossCoef_ridge","lossCoef_disHLL","lossCoef_Laplacian","watch_duration","score"]
    keys=["lossCoef_stab","lossCoef_ridge","lossCoef_disHLL","watch_duration","score"]

    for k in keys:
        fig,ax=plt.subplots()
        for family_trainer in family_trainers:
            family_trainer.plot_metric(k,ax)

    plt.show()
plot_history_famparams()

### Historique des trois `loss`

In [ ]:
def plot_history_losses():
    # keys=["loss_stab","loss_disHLL","loss_ridge","loss_Laplacian"]
    keys=["loss_stab","loss_disHLL","loss_ridge"]

    for k in keys:
        fig,ax=plt.subplots()
        for family_trainer in family_trainers:
            family_trainer.plot_metric(k,ax)

    plt.show()
plot_history_losses()

### Observation des famparams

In [ ]:
import tensorflow as tf
family_trainer=family_trainers[0]

for char in ['a', 'b', 'c', 'd', 'e', 'f']:
    best_agent=family_trainer.agents[char].agent
    print(char)
    print(best_agent.famparams)
    print(best_agent.agent_score())

best_agent=family_trainer.get_best_agent()
# best_agent=family_trainer.agents['a'].agent
print("best agent")
print(best_agent.famparams)
# print(best_agent.agent_score())


## Testons sur le jeu "loop" (périodique)

## Testons le long du temps

In [ ]:
def compare_along_the_time(kind):
    nb_t = 2000
    
    if kind==GenParam.kind_sod:
        param.BC_model=param.BC_solver=Param.BC_neumann
    genParam.kind=kind
    generator=FuncGenerator(genParam,param,k_tf32)
    W_val = core.generate_fine_solutions(genParam, param, nb_t, k_tf32)
    
    best_agent.load_WY_valid(W_val)

    res_fine_proj,res_HLL,res_model=best_agent.predict()
        
    nb_batch_plot=5
    if nb_batch_plot>W_val.shape[1]:
        nb_batch_plot=W_val.shape[1]
    nb_t_plot=4
    step=nb_t//nb_t_plot
    fig, ax = plt.subplots(nb_t_plot,nb_batch_plot,figsize=(15,nb_t_plot*2))
    if nb_batch_plot==1:ax=ax[:,tf.newaxis]
    for i in range(nb_t_plot):
        for j in range(nb_batch_plot):
            t=i*step
            ax[i,j].plot(res_fine_proj[t,j,:,0],color="k",label="fine")
            ax[i,j].plot(res_HLL[t,j,:,0],color="b",label="HLL")
            ax[i,j].plot(res_model[t,j,:,0],color="r",label="model")
    ax[0,0].legend()    

    fig.tight_layout()
    
    errors_HLL=tf.reduce_mean((res_fine_proj-res_HLL)**2,axis=[1,2,3])
    errors_model=tf.reduce_mean((res_fine_proj-res_model)**2,axis=[1,2,3])
    
    fig,ax=plt.subplots()
    ax.plot(errors_HLL,label="HLL")
    ax.plot(errors_model,label="model")

    if (np.max(errors_model)>0.1):
        ax.set_ylim(0,0.1)
    
    ax.legend()
    ax.set_xlabel("time")

    plt.show()

### kind Loop

In [ ]:
compare_along_the_time(GenParam.kind_loop)

### kind changing

In [ ]:
compare_along_the_time(GenParam.kind_changing)

### kind Sod

In [ ]:
compare_along_the_time(GenParam.kind_sod)

### Kind random Sod

In [ ]:
compare_along_the_time(GenParam.kind_random_sod)